In [1]:
#Basic imports
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from os import environ

#Classifier imports
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

#ML framework imports
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale

#Downsamplers imports - prototype generation
from imblearn.under_sampling import ClusterCentroids

#Downsamplers imports - prototype selection - controlled
from imblearn.under_sampling import RandomUnderSampler, NearMiss

#Downsamplers imports - prototype selection - Cleaning techniques
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours, RepeatedEditedNearestNeighbours

#Downsamplers imports - prototype selection - Cleaning techniques - Condensed nearest neighbors and derived algorithms
from imblearn.under_sampling import CondensedNearestNeighbour, OneSidedSelection, NeighbourhoodCleaningRule

#Downsamplers imports - prototype selection - Cleaning techniques
from imblearn.under_sampling import InstanceHardnessThreshold

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

ABSOLUTE_NEGATIVES = False
FILTER_DOMAIN = False
FILTER_MAX_SCORE_ZERO = False
out_dir = "mediode_stacking"

### Reading the input dataset

In [19]:
curr_dir = !pwd
input_path = curr_dir[0]+"/../domains_similarity/filtered_features_table/"
filename = "positions_features_mediode_filter_01.25.18.csv"

bind_scores_num = 10

#Features table
features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
features_cols = features_all.columns[1:-bind_scores_num] #removing binding scores and domain name
ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]
print "all samples positions #: "+str(features_all.shape[0])

#lignd binding domains dictionary
with open(curr_dir[0]+"/../ligands_negatives_domains_dict.pik", 'rb') as handle:
        negatives_dict = pickle.load(handle)
        
#1st-level predictions path
first_probs_path = curr_dir[0]+"/1st_level_pred/"

all samples positions #: 38944


#### Dataset of negative examples

In [3]:
def filter_to_ligand_binding_domains(use_max_binding_score):
    
    ligands_negatives_df = {}
    for ligand in ligands:
        
        ligands_negatives_df[ligand] = pd.DataFrame()
        for domain in negatives_dict[ligand].keys():
            if domain == 'negatives' or domain == 'domains':
                continue
            domain_all = features_all.loc[features_all.loc[:,"domain_name"] == domain,:]
            
            #In case this domain was previously filtered
            if len(domain_all) == 0:
                continue
            
            if (use_max_binding_score):
                ligands_negatives_df[ligand] = pd.concat([ligands_negatives_df[ligand],domain_all.loc[domain_all.loc[:,"max_binding_score"] == 0,:]])
            else:
                ligand_bind_str = ligand+"_binding_score"
                ligands_negatives_df[ligand] = pd.concat([ligands_negatives_df[ligand],domain_all.loc[domain_all.loc[:,ligand_bind_str] == 0,:]])
        
    #Handeling the ligand "all_ligands"
    all_ligands_negatives_df = pd.concat([ligands_negatives_df["dna"], ligands_negatives_df["dnabase"], ligands_negatives_df["dnabackbone"], ligands_negatives_df["rna"], ligands_negatives_df["rnabase"], 
                                 ligands_negatives_df["rnabackbone"], ligands_negatives_df["ion"], ligands_negatives_df["peptide"], ligands_negatives_df["metabolite"]])
    all_ligands_negatives_df = all_ligands_negatives_df.drop_duplicates()
    #Filter to just positions with max. binding score = 0
    all_ligands_negatives_df = all_ligands_negatives_df[all_ligands_negatives_df["max_binding_score"] == 0]
    ligands_negatives_df["all_ligands"] = all_ligands_negatives_df
    
    #Leaving just the features columns
    for ligand in ligands_negatives_df.keys():   
        ligands_negatives_df[ligand] = ligands_negatives_df[ligand][features_cols]
        print(ligand+" non-binding #:"+str(len(ligands_negatives_df[ligand])))
    
    return ligands_negatives_df

In [4]:
def negatives_by_binding_score(use_max_binding_score, filter_max_score_zero):
    
    ligands_negatives_df = {}
    for ligand in ligands:
        
        if use_max_binding_score:
            ligand_bind_str = "max_binding_score"
        else:
            ligand_bind_str = ligand+"_binding_score"
        
        if (filter_max_score_zero):
            ligands_negatives_df[ligand] = features_all[features_all[ligand_bind_str] == 0][features_all["max_binding_score"] != 0]
        else:
            ligands_negatives_df[ligand] = features_all[features_all[ligand_bind_str] == 0]
        ligands_negatives_df[ligand] = ligands_negatives_df[ligand].loc[:,features_cols]
        print(ligand+" non-binding #:"+str(len(ligands_negatives_df[ligand])))
        
    #Handeling the ligand "all_ligands"
    ligands_negatives_df["all_ligands"] = features_all[features_all["max_binding_score"] == 0]
    ligands_negatives_df["all_ligands"] = ligands_negatives_df["all_ligands"].loc[:,features_cols]
    print("all_ligands non-binding #:"+str(len(ligands_negatives_df["all_ligands"])))
    
    return ligands_negatives_df

In [5]:
#Create negatives datasets
if FILTER_DOMAIN:
    if ABSOLUTE_NEGATIVES:
        ligands_negatives_df = filter_to_ligand_binding_domains(True)
    else:
        ligands_negatives_df = filter_to_ligand_binding_domains(False)
else:
    if ABSOLUTE_NEGATIVES:
        ligands_negatives_df = negatives_by_binding_score(True, FILTER_MAX_SCORE_ZERO)
    else:
        ligands_negatives_df = negatives_by_binding_score(False, FILTER_MAX_SCORE_ZERO)

dna non-binding #:38095
dnabase non-binding #:38577
dnabackbone non-binding #:38203
rna non-binding #:38047
rnabase non-binding #:38407
rnabackbone non-binding #:38223
peptide non-binding #:35437
ion non-binding #:34488
metabolite non-binding #:33971
all_ligands non-binding #:27191


#### Datasets of positive examples by ligand

In [6]:
bind_th = 0.1
ligands_features_df = {}

for ligand in ligands:
    score_col_str = ligand+"_binding_score"
    ligand_binding_df = features_all[features_all[score_col_str] >= bind_th]
    print ligand+" #: "+str(ligand_binding_df.shape[0])
    ligands_features_df[ligand] = ligand_binding_df.loc[:,features_cols]

dna #: 501
dnabase #: 193
dnabackbone #: 408
rna #: 433
rnabase #: 224
rnabackbone #: 308
peptide #: 1496
ion #: 1093
metabolite #: 1525


#### Dataset of positive examples - all ligands combined

In [7]:
all_ligands_features_df = pd.concat([ligands_features_df["dna"], ligands_features_df["dnabase"], ligands_features_df["dnabackbone"], ligands_features_df["rna"], ligands_features_df["rnabase"], 
                                     ligands_features_df["rnabackbone"], ligands_features_df["ion"], ligands_features_df["peptide"], ligands_features_df["metabolite"]])
all_ligands_features_df = all_ligands_features_df.drop_duplicates()
print "all_ligands #: "+str(all_ligands_features_df.shape[0])
ligands_features_df["all_ligands"] = all_ligands_features_df

all_ligands #: 4518


### Models tested (and their hyper-parameters)

In [583]:
classifiers = {}
classifiers["Logistic"] = LogisticRegression(C=0.001, random_state=0)
classifiers["RF"] = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)  
#classifiers["RF"] = RandomForestRegressor(n_estimators=1000)  
classifiers["KNN"] = KNeighborsClassifier(n_neighbors=100, n_jobs=-1)
#classifiers["KNN"] = KNeighborsRegressor(n_neighbors=100)
classifiers["SVM"] = SVC(kernel='rbf', probability=True, random_state=0)
classifiers["ADA-RF"] = AdaBoostClassifier(n_estimators=1000, random_state=0)
classifiers["ADA-Log"] = AdaBoostClassifier(base_estimator=classifiers["Logistic"], n_estimators=1000, random_state=0)
classifiers["Bag-Log"] = BaggingClassifier(base_estimator=classifiers["Logistic"], n_estimators=1000, n_jobs=-1, random_state=0)
classifiers["XGB"] = XGBClassifier(n_estimators=100, n_jobs=-1, random_state=0, max_depth=2, min_child_weight=4, colsample_bytree=0.75)

### Downsamplers tested

In [10]:
#documentation on techniques: http://contrib.scikit-learn.org/imbalanced-learn/stable/under_sampling.html#cleaning-under-sampling-techniques
downsamplers = defaultdict(dict)

##Prototype generation##
downsamplers["ClusterCentroids"] = ClusterCentroids(random_state=0)

##Prototype selection##
#Contolled#
downsamplers["RandomUnderSampler"] = RandomUnderSampler(random_state=0)
downsamplers["NearMiss3"] = NearMiss(random_state=0, version=3)
downsamplers["NearMiss2"] = NearMiss(random_state=0, version=2)
downsamplers["NearMiss1"] = NearMiss(random_state=0, version=1)

#Cleaning#
downsamplers["TomekLinks"] = TomekLinks(random_state=0)
downsamplers["EditedNearestNeighbours"] = EditedNearestNeighbours(random_state=0)
downsamplers["RepeatedEditedNearestNeighbours"] = RepeatedEditedNearestNeighbours(random_state=0)
downsamplers["NeighbourhoodCleaningRule"] = NeighbourhoodCleaningRule(random_state=0)

# Instance hardness threshold#
downsamplers["InstanceHardnessThreshold"]["KNN"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["KNN"])
#downsamplers["InstanceHardnessThreshold"]["KNN"] = InstanceHardnessThreshold(random_state=0, estimator= KNeighborsClassifier(n_neighbors=100))
downsamplers["InstanceHardnessThreshold"]["SVM"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["SVM"])
downsamplers["InstanceHardnessThreshold"]["RF"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["RF"])
#downsamplers["InstanceHardnessThreshold"]["RF"] = InstanceHardnessThreshold(random_state=0, estimator=RandomForestClassifier(n_estimators=1000))
downsamplers["InstanceHardnessThreshold"]["Logistic"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["Logistic"])
downsamplers["InstanceHardnessThreshold"]["ADA-RF"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["ADA-RF"])
downsamplers["InstanceHardnessThreshold"]["ADA-Log"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["ADA-Log"])
downsamplers["InstanceHardnessThreshold"]["Bag-Log"] = InstanceHardnessThreshold(random_state=0, estimator=classifiers["Bag-Log"])

### Reading env input for downsampler technique, ligand and classifier

In [11]:
#Reading the ligand input
try:
    ligand = environ['ligand']
except:
    ligand = "dnabase"
print "ligand = "+ligand
    
#Reading the downsampler input
try: 
    downsample_method = environ['down']
except:
    downsample_method = "RandomUnderSampler"
print "downsample_method = "+downsample_method

#Reading the classifier input
try: 
    classifier_method = environ['classifier']
except:
    classifier_method = "Logistic"
print "classifier_method = "+classifier_method

ligand = dnabase
downsample_method = RandomUnderSampler
classifier_method = Logistic


### Test model functions

In [363]:
def test_model(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_negatives_features, ligand_name, downsample_method, include_other_ligands, features=[]):
    """
    Test different models in 10-folds cross-validation.
    """
    models_included = ["XGB_prob", "RF_prob", "ADA_prob", "SVM_prob", "KNN_prob"]
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        classifier = classifier_method
        model = classifiers[classifier]
        features_pred_dfs[classifier] = pd.DataFrame()
        
        #Create X and y with included features
        X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
        
        if (classifier in models_req_scaling):
            idx = X.index
            cols = X.columns
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?
            X.index = idx #Restoring indices after scaling
            X.columns = cols

        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * ligand_negatives_features.shape[0])
        y = np.array(y)

        binding_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
        pred_idx = 1

        for train_index, test_index in binding_skf.split(X, y):
            print "fold #: "+str(pred_idx)
            X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #Down-sample negative examples to have balanced classes
            if (downsample_method == "NoDown"):
                X_train_sampled = X_train
                y_train_sampled = y_train
            else:
                if (downsample_method == "InstanceHardnessThreshold"):
                    downsampler = downsamplers[downsample_method][classifier]
                else:
                    downsampler = downsamplers[downsample_method]

                X_train_sampled, y_train_sampled = downsampler.fit_sample(X_train, y_train)
            
            #Get relevant out-of-fold tables
            oof_1st_probs = pd.read_csv(first_probs_path+str(pred_idx)+"/"+ligand_name+"_dataset_probs.csv", sep='\t', index_col=0)
            
            #Create the features table for training using the 1st level prediction probs
            X_train_probs_features = X_train_sampled.merge(oof_1st_probs, how="left", left_index=True, right_index=True)
            del X_train_probs_features["fold"]
            X_train_probs = X_train_probs_features[models_included]
            print "X_train_probs.shape #:"+str(X_train_probs.shape)
            
            #Add features from other ligands predictions
            if (include_other_ligands == True):
                for other_ligand in ligands:
                    if (other_ligand == ligand_name):
                        continue
                    #get the relevant out-of-fold tables
                    oof_dataset_probs = pd.read_csv(first_probs_path+str(pred_idx)+"/"+other_ligand+"_dataset_probs.csv", sep='\t', index_col=0)
                    oof_between_probs = pd.read_csv(first_probs_path+str(pred_idx)+"/"+other_ligand+"_between_probs.csv", sep='\t', index_col=0)
                    oof_other_ligand_probs = pd.concat([oof_dataset_probs, oof_between_probs])[models_included]
                    oof_cols = [x+"_"+other_ligand for x in oof_other_ligand_probs.columns]
                    oof_other_ligand_probs.columns = oof_cols
                    
                    #Add the features relevant to the training samples
                    X_train_probs = X_train_probs.merge(oof_other_ligand_probs, how="left", left_index=True, right_index=True)
                print "all ligands X_train_probs.shape #:"+str(X_train_probs.shape)
                    
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_probs, y_train_sampled)
            probs_list = []

            #Create the features table of testing using the 1st level prediction probs
            X_test_probs_features = X_test.merge(oof_1st_probs, how="left", left_index=True, right_index=True)
            del X_test_probs_features["fold"]
            X_test_probs = X_test_probs_features[models_included]
            print "X_test_probs.shape #:"+str(X_test_probs.shape)
            
            if (include_other_ligands == True):
                for other_ligand in ligands:
                    if (other_ligand == ligand_name):
                        continue
                    #get the relevant out-of-fold tables
                    oof_dataset_probs = pd.read_csv(first_probs_path+str(pred_idx)+"/"+other_ligand+"_dataset_probs.csv", sep='\t', index_col=0)
                    oof_between_probs = pd.read_csv(first_probs_path+str(pred_idx)+"/"+other_ligand+"_between_probs.csv", sep='\t', index_col=0)
                    oof_other_ligand_probs = pd.concat([oof_dataset_probs, oof_between_probs])[models_included]
                    oof_cols = [x+"_"+other_ligand for x in oof_other_ligand_probs.columns]
                    oof_other_ligand_probs.columns = oof_cols
                    
                    #Add the features relevant to the training samples
                    X_test_probs = X_test_probs.merge(oof_other_ligand_probs, how="left", left_index=True, right_index=True)
                print "all ligands X_test_probs.shape #:"+str(X_test_probs.shape)
                    
            probs = model.predict_proba(X_test_probs)
            for l in probs:
                probs_list.append(l[1])
                
            pred_dict["obs"].extend(y_test)
            pred_dict["prob"].extend(probs_list)
            fold_list = [pred_idx] * len(probs_list)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(probs_list)
            pred_dict["model"].extend(model_list)

            #Update auc auprc dictionaries
            auc_dict[classifier].append(roc_auc_score(y_test, probs[:, 1]))
            precision, recall, _ = precision_recall_curve(y_test, probs[:, 1])
            auprc_dict[classifier].append(auc(recall, precision))
            
            #Update features table
            features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test_probs_features)
            pred_idx += 1
            
            print "AUC = "+str(auc_dict[classifier][-1])
            print "AUPRC = "+str(auprc_dict[classifier][-1])

        avg_auc = np.sum(auc_dict[classifier])/10.0
        print "avg auc = "+str(avg_auc)
        
        avg_auprc = np.sum(auprc_dict[classifier])/10.0
        print "avg auprc = "+str(avg_auprc)
            
        print "Finished "+ligand+" "+classifier
        break
    
    return features_pred_dfs

In [396]:
def combine_features_predictions(ligand, ordered_features, pred_df):
    
    pred_res = pred_df.copy(deep=True)
    for classifier in classifiers.keys():
        classifier = classifier_method
        model_pred = pred_res[pred_res["model"] == classifier]
        model_pred.index = ordered_features[classifier].index
        
        #Creating the combined table
        features_pred = pd.concat([ordered_features[classifier], model_pred], axis=1)
        
        #Saving
        features_pred.to_csv(curr_dir[0]+"/2nd_level_pred/just_probs_one_ligand/features_pred_tables/"+ligand+"_"+classifier+"_features_pred.csv", sep='\t')
        break

#### Predict for each ligand seperatelly

In [585]:
%%time
classifier_method = "XGB"
downsample_method = "NoDown"
ligand = "metabolite"
include_other_ligands = False
save_to_file = True

#Initialize dictionary
pred_dict = defaultdict(list)
auc_dict = defaultdict(list)
auprc_dict = defaultdict(list)

ordered_features = test_model(pred_dict, auc_dict, auprc_dict, ligands_features_df[ligand], ligands_negatives_df[ligand], ligand, downsample_method, include_other_ligands)

pred_df = pd.DataFrame.from_dict(pred_dict)
auc_df = pd.DataFrame.from_dict(auc_dict)
auprc_df = pd.DataFrame.from_dict(auprc_dict)

if (save_to_file):
    #Save to file
    pred_df.to_csv(curr_dir[0]+"/2nd_level_pred/just_probs_one_ligand/"+ligand+"_"+classifier_method+"_0.1.csv", sep='\t')
    auc_df.to_csv(curr_dir[0]+"/2nd_level_pred/just_probs_one_ligand/"+ligand+"_"+classifier_method+"_0.1_auc.csv", sep='\t')
    auprc_df.to_csv(curr_dir[0]+"/2nd_level_pred/just_probs_one_ligand/"+ligand+"_"+classifier_method+"_0.1_auprc.csv", sep='\t')

    #Combine features and pred results to a unified table
    combine_features_predictions(ligand, ordered_features, pred_df)

print "Finished ligand "+ligand

fold #: 1
X_train_probs.shape #:(31945, 5)
X_test_probs.shape #:(3551, 5)
AUC = 0.791329578722
AUPRC = 0.214275799836
fold #: 2
X_train_probs.shape #:(31946, 5)
X_test_probs.shape #:(3550, 5)
AUC = 0.786902514906
AUPRC = 0.187796362149
fold #: 3
X_train_probs.shape #:(31946, 5)
X_test_probs.shape #:(3550, 5)
AUC = 0.831059123679
AUPRC = 0.252674817122
fold #: 4
X_train_probs.shape #:(31946, 5)
X_test_probs.shape #:(3550, 5)
AUC = 0.818005506589
AUPRC = 0.2557645156
fold #: 5
X_train_probs.shape #:(31946, 5)
X_test_probs.shape #:(3550, 5)
AUC = 0.824528948072
AUPRC = 0.229260543644
fold #: 6
X_train_probs.shape #:(31947, 5)
X_test_probs.shape #:(3549, 5)
AUC = 0.818813620377
AUPRC = 0.235439062025
fold #: 7
X_train_probs.shape #:(31947, 5)
X_test_probs.shape #:(3549, 5)
AUC = 0.804544063647
AUPRC = 0.181499342046
fold #: 8
X_train_probs.shape #:(31947, 5)
X_test_probs.shape #:(3549, 5)
AUC = 0.794167260586
AUPRC = 0.229200279939
fold #: 9
X_train_probs.shape #:(31947, 5)
X_test_probs.sh